# Padova Grand Tour - Queries

This notebook will provide ten insightful queries about our data. Get started by starting GraphDB with `docker compose up --force-recreate`.


Sparql wrapper (copy and pasted from the Individual Project's notebooks):

In [2]:

from SPARQLWrapper import SPARQLWrapper, JSON

# Taken from `data/ttlData/ontology.ttl`
prefixString = """
PREFIX ofn: <http://www.ontotext.com/sparql/functions/>
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX pgt: <https://padovagrandtour.github.io/entities#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX xml: <http://www.w3.org/XML/1998/namespace> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
PREFIX sdo: <https://schema.org/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    # Our local endpoint, the repo name is defined in `data/graphdb-repo.ttl` 
    sparql = SPARQLWrapper("http://localhost:7210/repositories/pgt") 
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            array.append(app)
        print(len(array)) # Print the array length
        return array

    except Exception as e :
        print("The operation failed", e)
    

In [3]:
# Improved version that also handles prefixes
mappedPrefixes = {}
for prefix in prefixString.split('\n'):
    if('PREFIX' in prefix):
        prefix = prefix.replace('PREFIX', '', 1).split(':', 1)
        mappedPrefixes[prefix[0].strip()] = prefix[1].strip().replace('<','').replace('>','')

def query(queryString, replacePrefixes=True):
    queryResults = run_query(queryString)
    if(replacePrefixes):
        for queryIndex, queryResultRaw in enumerate(queryResults):
            for subQueryIndex, subQueryResultRaw in enumerate(queryResultRaw):
                queryResult = queryResultRaw[subQueryIndex][1]
                for prefixName, prefixValue in mappedPrefixes.items():
                    queryResult = queryResult.replace(prefixValue, prefixName + ':')

                queryResults[queryIndex][subQueryIndex] = (queryResultRaw[subQueryIndex][0], queryResult)

        return queryResults
    else: return queryResults


def insertQuery(queryString):
    to_run = prefixString + "\n" + queryString

    # Our local endpoint, the repo name is defined in `data/graphdb-repo.ttl` 
    sparql = SPARQLWrapper("http://localhost:7210/repositories/pgt/statements") 
    sparql.setTimeout(300)
    sparql.method = "POST"
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)    
    sparql.queryType = "INSERT"
    results = sparql.query()
    return results.convert()


Check if everything works:

In [4]:
query("""
select distinct * {
    ?s ?p ?o
}
LIMIT 3
""", replacePrefixes=True)


3


[[('s', 'rdf:type'), ('p', 'rdf:type'), ('o', 'rdf:Property')],
 [('s', 'rdfs:subPropertyOf'), ('p', 'rdf:type'), ('o', 'rdf:Property')],
 [('s', 'rdfs:subPropertyOf'),
  ('p', 'rdf:type'),
  ('o', 'owl:TransitiveProperty')]]

## 1 - Get tour sites in order

Let's start by .
We would like to retrieve every site touched by a specific tour, with name and coordinates.

If you just follow a naive approach, this is what you get:

```
[[('stepIndex', 'rdf:_1'), ('site', 'pgt:SITE1')],
 [('stepIndex', 'rdf:_10'), ('site', 'pgt:SITE10')],
 [('stepIndex', 'rdf:_11'), ('site', 'pgt:SITE11')],
 [('stepIndex', 'rdf:_12'), ('site', 'pgt:SITE12')],
 [('stepIndex', 'rdf:_13'), ('site', 'pgt:SITE13')],
 [('stepIndex', 'rdf:_14'), ('site', 'pgt:SITE14')],
 [('stepIndex', 'rdf:_15'), ('site', 'pgt:SITE15')],
 [('stepIndex', 'rdf:_2'), ('site', 'pgt:SITE2')],
 [('stepIndex', 'rdf:_3'), ('site', 'pgt:SITE3')],
 [('stepIndex', 'rdf:_4'), ('site', 'pgt:SITE4')],
 [('stepIndex', 'rdf:_5'), ('site', 'pgt:SITE5')],
 [('stepIndex', 'rdf:_6'), ('site', 'pgt:SITE6')],
 [('stepIndex', 'rdf:_7'), ('site', 'pgt:SITE7')],
 [('stepIndex', 'rdf:_8'), ('site', 'pgt:SITE8')],
 [('stepIndex', 'rdf:_9'), ('site', 'pgt:SITE9')]]
 ```

 This happens because `rdf:_15` has a lower lexicographical order than `rdf:_2`. To get the right order, we need to strip the `rdf:` prefix and cast the result to integer.

 We do not have name/lat/long for every site, some information are missing. To account for that, we use the `OPTIONAL` keyword.

In [5]:
tourName = "Tour 1"
query('''
SELECT ?stepIndexNumber ?siteName ?siteLat ?siteLong WHERE {
    ?tour a pgt:Tour;
          sdo:name "''' + tourName + '''";
	      pgt:steps ?stepNode .
    ?stepNode ?stepIndex ?site .

    FILTER (strstarts(str(?stepIndex), 'http://www.w3.org/1999/02/22-rdf-syntax-ns#_'))

    BIND (STRLEN("http://www.w3.org/1999/02/22-rdf-syntax-ns#_") AS ?prefixLength)
    BIND (xsd:integer(SUBSTR(xsd:string(?stepIndex),?prefixLength + 1)) AS ?stepIndexNumber).

    OPTIONAL{ ?site sdo:name ?siteName. }
    OPTIONAL{ ?site geo:lat  ?siteLat.  }
    OPTIONAL{ ?site geo:long ?siteLong. }


} ORDER BY ?stepIndexNumber

''')


15


[[('stepIndexNumber', '1'),
  ('siteName', 'Museo archeologico'),
  ('siteLat', '45.41096111111111'),
  ('siteLong', '11.880011111111111')],
 [('stepIndexNumber', '2'),
  ('siteName', "Museo d'Arte Medievale e Moderna"),
  ('siteLat', '45.41096111111111'),
  ('siteLong', '11.880011111111111')],
 [('stepIndexNumber', '3'),
  ('siteName', 'Museo Bottacin'),
  ('siteLat', '45.411569444444446'),
  ('siteLong', '11.878119444444446')],
 [('stepIndexNumber', '4'),
  ('siteName', "Museo del Risorgimento e dell'Età Contemporanea"),
  ('siteLat', '45.40784722222222'),
  ('siteLong', '11.876925')],
 [('stepIndexNumber', '5'),
  ('siteName', 'Museo di anatomia patologica (Padova)'),
  ('siteLat', '45.405027777777775'),
  ('siteLong', '11.8859')],
 [('stepIndexNumber', '6'),
  ('siteName', 'Centro di Ateneo per i Musei - CAM')],
 [('stepIndexNumber', '7'),
  ('siteName', 'Collezione privata Safilo'),
  ('siteLat', '45.414833333333334'),
  ('siteLong', '11.928894444444444')],
 [('stepIndexNumber', '

## 2 - Check for missing information

In the previous query, we noticed some data is missing from a bunch of cultural sites. What if we want to check for missing data or orphan nodes?

We may do this via sparQL, and use the `UNION` keyword to get all the errors at once.

In [6]:
query('''
SELECT DISTINCT ?error ?entity WHERE {


    { 
        SELECT ?error (?site AS ?entity)  WHERE { 
            BIND( "Missing geo data" as ?error)
            ?site a pgt:CulturalSite .
            FILTER NOT EXISTS {
                ?site geo:lat ?lat;
                    geo:long ?long.
            }
        }
    } UNION { 
        SELECT ?error (?site AS ?entity)  WHERE { 
            BIND( "Missing CulturalSiteCategory" as ?error)
            ?site a pgt:CulturalSite .
            FILTER NOT EXISTS {
                ?artwork pgt:hasSite ?site;
            }
        }
    } UNION { 
        SELECT ?error (?artwork AS ?entity)  WHERE { 
            BIND( "Missing CulturalSite" as ?error)
            ?artwork a pgt:Artwork .
            FILTER NOT EXISTS {
                ?artwork pgt:hasSite ?site;
            }
        }
    } 


}
''')



184


[[('error', 'Missing geo data'), ('entity', 'pgt:SITE63')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE41')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE45')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE13')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE6')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE105')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE106')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE108')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE110')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE112')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE114')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE116')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE117')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE119')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE133')],
 [('error', 'Missing geo data'), ('entity', 'pgt:SITE134')],
 [('error', 'Missing geo data'

## 3 - Get meaningful (with coordinates) sites by number of artworks

Now we would like to create a new tour. In order to do so, we would like to see which sites have associated artwork, so we may pick meaningful sites.


In [7]:
query(''' 
SELECT DISTINCT ?name (COUNT(?artwork) as ?number) WHERE { 
    ?artwork a pgt:Artwork; 
            pgt:hasSite ?site. 
    ?site sdo:name ?name. 
        FILTER EXISTS {?site geo:lat ?lat} 
        FILTER EXISTS {?site geo:long ?long} 
 
    }   
GROUP BY ?name  
ORDER BY DESC(?number) 
LIMIT 100 
''')

12


[[('name', 'Palazzina del prefetto (ex)'), ('number', '804')],
 [('name', 'Convento degli Eremitani'), ('number', '752')],
 [('name', 'Complesso Cavalli'), ('number', '559')],
 [('name', 'Palazzo Folco'), ('number', '468')],
 [('name', "Museo d'Arte Medievale e Moderna"), ('number', '358')],
 [('name', 'Museo archeologico'), ('number', '279')],
 [('name', 'Palazzo ECA (ex)'), ('number', '179')],
 [('name', 'Cappella degli Scrovegni'), ('number', '68')],
 [('name', 'Museo Bottacin'), ('number', '28')],
 [('name', 'Palazzo Wollemborg'), ('number', '20')],
 [('name', 'Complesso Ingegneria'), ('number', '10')],
 [('name', "Museo del Risorgimento e dell'Età Contemporanea"),
  ('number', '10')]]

As you see, artworks are splitted across very few sites compared to the one we have, which are:

In [8]:
query('''
SELECT DISTINCT (COUNT(?site) as ?numberOfCulturalSites) WHERE {
    ?site a pgt:CulturalSite;
} LIMIT 1
''')

1


[[('numberOfCulturalSites', '159')]]

This may be explained considering that:
- many sites are not museums, but squared or other monuments
- many sites do not provide data about their artworks

## 4 - Get sites by distance from a point

https://graphdb.ontotext.com/documentation/9.6/free/using-math-functions-with-sparql.html



In [9]:
# Python 3 program to calculate Distance Between Two Points on Earth
from math import radians, cos, sin, asin, sqrt

def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers = 6371. Use 3956 for miles
    # Since we have small distances, I've changed it to meters
    r = 6371 * 1000
      
    # calculate the result
    return(c * r)

def distance2(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = lon1 * 3.14159265359 / 180
    lon2 = lon2 * 3.14159265359 / 180
    lat1 = lat1 * 3.14159265359 / 180
    lat2 = lat2 * 3.14159265359 / 180
      

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    print(lat1, lat2, lon1, lon2, dlat, dlon)

    return 2 * asin(sqrt(sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2)) * 6371 * 1000
 
    
    

'''
 [('stepIndexNumber', '2'),
  ('siteName', "Museo d'Arte Medievale e Moderna"),
  ('siteLat', '45.41096111111111'),
  ('siteLong', '11.880011111111111')],
 [('stepIndexNumber', '3'),
  ('siteName', 'Museo Bottacin'),
  ('siteLat', '45.411569444444446'),
  ('siteLong', '11.878119444444446')],
'''
# driver code
lat1 = 45.41096111111111
lat2 = 45.411569444444446
lon1 = 11.880011111111111
lon2 =  11.878119444444446
print(distance(lat1, lat2, lon1, lon2), "m")
print(distance2(lat1, lat2, lon1, lon2), "m")


query('''
SELECT DISTINCT ?lat1 ?lat2 ?lon1 ?lon2 ?dlat ?dlon ?distance {
    BIND("''' + str(lat1) +'''"^^xsd:float * 3.14159265359 / 180 as ?lat1)
    BIND("''' + str(lat2) +'''"^^xsd:float * 3.14159265359 / 180 as ?lat2)
    BIND("''' + str(lon1) +'''"^^xsd:float * 3.14159265359 / 180 as ?lon1)
    BIND("''' + str(lon2) +'''"^^xsd:float * 3.14159265359 / 180 as ?lon2)
    BIND(?lat2 - ?lat1 AS ?dlat)
    BIND(?lon2 - ?lon1 AS ?dlon)

    BIND(2 * ofn:asin(ofn:sqrt(ofn:sin(?dlat / 2)* ofn:sin(?dlat / 2) + ofn:cos(?lat1) * ofn:cos(?lat2) * ofn:sin(?dlon / 2) * ofn:sin(?dlon / 2))) * 6371 * 1000 AS ?distance)

}
''')


162.42020990254528 m
0.7925707878840436 0.7925814053036598 0.20734530906241244 0.20731229325072892 1.0617419616276713e-05 -3.30158116835233e-05
162.42020990224358 m
1


[[('lat1', '0.7925708'),
  ('lat2', '0.79258144'),
  ('lon1', '0.20734532'),
  ('lon2', '0.20731232'),
  ('dlat', '1.0609627E-5'),
  ('dlon', '-3.3006072E-5'),
  ('distance', '162.35992852325901')]]

Query:

In [10]:
lat = 45.411569444444446
lon = 11.880011111111111
maxDistance = 3000 # meters

query('''
SELECT DISTINCT ?siteName ?distance {
    BIND("''' + str(lat) +'''"^^xsd:float * 3.14159265359 / 180 as ?lat1)
    BIND("''' + str(lon) +'''"^^xsd:float * 3.14159265359 / 180 as ?lon1)

    ?site a pgt:CulturalSite .
    ?site sdo:name ?siteName .

    ?site geo:lat  ?lat2deg .
    ?site geo:long ?lon2deg .


    BIND(?lat2deg * 3.14159265359 / 180 as ?lat2)
    BIND(?lon2deg * 3.14159265359 / 180 as ?lon2)
    BIND(?lat2 - ?lat1 AS ?dlat)
    BIND(?lon2 - ?lon1 AS ?dlon)

    BIND(2 * ofn:asin(ofn:sqrt(ofn:sin(?dlat / 2)* ofn:sin(?dlat / 2) + ofn:cos(?lat1) * ofn:cos(?lat2) * ofn:sin(?dlon / 2) * ofn:sin(?dlon / 2))) * 6371 * 1000 AS ?distance)

    FILTER(?distance < "''' + str(maxDistance) + '''"^^xsd:float).
}ORDER BY ?distance LIMIT 1000
''')

105


[[('siteName', 'Cappella degli Scrovegni'), ('distance', '46.1037033728453')],
 [('siteName', 'Museo archeologico'), ('distance', '67.59393234279999')],
 [('siteName', "Museo d'Arte Medievale e Moderna"),
  ('distance', '67.59393234279999')],
 [('siteName', 'Convento degli Eremitani'),
  ('distance', '144.38608405467824')],
 [('siteName', 'Palazzo Zuckermann'), ('distance', '147.61968528552447')],
 [('siteName', 'Museo Bottacin'), ('distance', '147.61968528552447')],
 [('siteName', 'Complesso Cavalli'), ('distance', '157.8335972843654')],
 [('siteName', 'Palazzo Cavalli alle Porte Contarine'),
  ('distance', '159.12098280757957')],
 [('siteName', 'Porte Contarine'), ('distance', '200.9640879391254')],
 [('siteName', 'Porta Altinate'), ('distance', '320.81427483311955')],
 [('siteName', 'Centro culturale Altinate/San Gaetano'),
  ('distance', '345.47557984269116')],
 [('siteName', 'Chiesa di San Gaetano (Padova)'),
  ('distance', '383.76138408772346')],
 [('siteName', 'Chiesa di San Fer

## 5 - Get the tallest artwork made by each author

In [11]:
query(''' 
SELECT DISTINCT ?author ?artworkName ?height WHERE { 
    ?artwork a pgt:Artwork; 
             sdo:author ?author; 
             pgt:height ?height. 
    FILTER (?height=?maxHeight && ?author = ?Innerauthor) 
    { 
 
        SELECT ?Innerauthor (MAX(?height) AS ?maxHeight) WHERE { 
                ?artwork a pgt:Artwork; 
                        sdo:author ?Innerauthor;  
                        pgt:height ?height. 
        }  
        GROUP BY (?Innerauthor) 
    } 
} 
 
''')

100


[[('site', 'pgt:SITE2'),
  ('material',
   '5 diamanti; oro; rubino ottagonale; 2 smeraldi ottagonali; taglio a rosetta')],
 [('site', 'pgt:SITE2'),
  ('material',
   '6 diamanti; oro; 4 rubini; smalto nero; smalto trasparente rosso-arancio; turchese; sfaccettatura; smaltatura; taglio a rosetta')],
 [('site', 'pgt:SITE3'), ('material', 'ARGENTO; CONIAZIONE')],
 [('site', 'pgt:SITE3'), ('material', 'ARGENTO; FUSIONE')],
 [('site', 'pgt:SITE0'), ('material', 'Affresco')],
 [('site', 'pgt:SITE1'),
  ('material', 'Agata a due strati, avori, grigio / incisione')],
 [('site', 'pgt:SITE1'),
  ('material',
   'Agata a tre strati, grigia, marrone pallido, marrone scuro / incisione')],
 [('site', 'pgt:SITE1'),
  ('material',
   'Agata a tre strati, grigio molto pallido, marrone scuro / incisione')],
 [('site', 'pgt:SITE2'),
  ('material', 'Agata corniola ovale; oro; taglio a piastra')],
 [('site', 'pgt:SITE1'), ('material', 'Agata semiopaca, zonata / incisione')],
 [('site', 'pgt:SITE1'),
  ('ma

## 6 - Generate a new tour

This query will generate a tour given the name, description and visited sites (as a list of names of cultural sites).

This is more a proof of concept than an actual useful query, since it would make way more sense to use a programming language to generate part of the query (for example, the UUID could be generated in Python instead) instead of a pure sparQL call.

You may safely run this query multiple times: thanks to the `BIND` + `MINUS` keywords, the insertion is skipped if a tour with the same name already exists.

*NOTE*: GraphDB requires a different endpoint for `INSERT` queries, thus we had to write a different function to handle inserts from the Notebook

In [12]:
tourName = "Generated Tour"
tourDescription = "Description of generated tour"

# It would be nice to avoid manual index assigning of sites' indexes,
# however, this is not possible, since sparQL lists are always unordered ones.
tourSteps = '''
    (1 "Chiesa di San Bonaventura delle Eremite")
    (2 "Battistero di Padova")
'''

insertQuery('''
INSERT { 
    ?tourID a pgt:Tour.
    ?tourStepsID a rdf:Seq.

    ?tourID pgt:steps ?tourStepsID.
    ?tourID sdo:name ?tourName.
    ?tourID sdo:description ?tourDescription.
    
    ?tourStepsID ?isStep ?stepSite.

} WHERE { 
    BIND("'''+ tourName + '''" as ?tourName).
    BIND("'''+ tourDescription + '''" as ?tourDescription).
    MINUS { 
        ?checkingTour a pgt:Tour;
           sdo:name ?tourName.
    }.
    BIND(StrUUID() AS ?uuidCode) . 
    BIND(URI(CONCAT("https://padovagrandtour.github.io/entities#TOUR-", ?uuidCode)) AS ?tourID).
    BIND(URI(CONCAT("https://padovagrandtour.github.io/entities#TOURSTEPS-", ?uuidCode)) AS ?tourStepsID).
    
    
    VALUES (?stepIndex ?stepName) {'''
        + tourSteps +
    '''}
    
    BIND(URI(CONCAT("http://www.w3.org/1999/02/22-rdf-syntax-ns#_", STR(?stepIndex))) AS ?isStep).
    ?stepSite a pgt:CulturalSite;
          sdo:name ?stepName.



}

''')

b''

## 7 - Get a tour preview

Getting some random artworks of what we are going to see in the newly generated tour.

In [13]:
query('''
SELECT ?siteName ?artworkName ?artworkDescription ?artworkImage ?artworkURL WHERE {
    ?tour a pgt:Tour;
          sdo:name "'''+ tourName + '''";
	      pgt:steps ?stepNode .
          
    ?stepNode ?stepIndex ?site .
    FILTER (strstarts(str(?stepIndex), 'http://www.w3.org/1999/02/22-rdf-syntax-ns#_'))


    ?site sdo:name ?siteName .

    ?artwork pgt:hasSite ?site;
             sdo:image ?artworkImage;
             sdo:name ?artworkName;
             sdo:url ?artworkURL;
             sdo:description ?artworkDescription.

} ORDER BY RAND() LIMIT 10
''')

Empty


[]

## 8 - Get tour recap/metrics

In this query we get some metrics about the whole tour, such as the total travelled distance and the average year of the artwork we are going to see.
First part of the query

In [ ]:
query('''


SELECT  ?distance ?site1 ?site2 WHERE {
    ?tour a pgt:Tour ;
          sdo:name "Tour 1" ;
	      pgt:steps ?stepNode .
          
    ?stepNode ?stepIndex ?startPoint ;
              ?stepIndex ?endPoint .
    FILTER (strstarts(str(?stepIndex), 'http://www.w3.org/1999/02/22-rdf-syntax-ns#_'))

    ?startPoint a pgt:CulturalSite ;
             sdo:name ?site1 ;
             geo:lat  ?lat1deg ;
             geo:lat  ?lon1deg .

    ?site2   geo:lat  ?lat2deg ;
             geo:lat  ?lon2deg .

    BIND(?lat1deg * 3.14159265359 / 180 as ?lat1)
    BIND(?lon1deg * 3.14159265359 / 180 as ?lon1)
    BIND(?lat2deg * 3.14159265359 / 180 as ?lat2)
    BIND(?lon2deg * 3.14159265359 / 180 as ?lon2)
    BIND(?lat2 - ?lat1 AS ?dlat)
    BIND(?lon2 - ?lon1 AS ?dlon)

    BIND(2 * ofn:asin(ofn:sqrt(ofn:sin(?dlat / 2)* ofn:sin(?dlat / 2) + ofn:cos(?lat1) * ofn:cos(?lat2) * ofn:sin(?dlon / 2) * ofn:sin(?dlon / 2))) * 6371 * 1000 AS ?distance)

    SELECT ?site2 {
        ?stepNode ?stepIndex ?EndPoint .
        FILTER (strstarts(str(?stepIndex), 'http://www.w3.org/1999/02/22-rdf-syntax-ns#_'))

        ?endPoint a pgt:CulturalSite ;
                sdo:name ?site2 ;
    } 
} FILTER 
''') 

2nd part of the query

In [ ]:
query('''
SELECT ?year  WHERE {
    ?tour a pgt:Tour ;
          sdo:name "Tour 1" ;
	        pgt:steps ?stepNode .
          
    ?stepNode ?stepIndex ?site .
    FILTER (strstarts(str(?stepIndex), 'http://www.w3.org/1999/02/22-rdf-syntax-ns#_'))
    
    ?artwork pgt:hasSite ?site ;
             pgt:yearCreatedText ?year ;
  } LIMIT 100
''') 

## 9 - Get cultural sites which have more than 50 artworks in poor state, displaying these artworks names

In [ ]:


query('''  
    SELECT ?site (COUNT(?artwork) AS ?artwork_count)   
    (GROUP_CONCAT(?artwork_name; SEPARATOR=", ") AS ?artwork_names)  
  
    WHERE {  
      ?site a pgt:CulturalSite .  
      ?artwork pgt:hasSite ?site.  
      ?artwork sdo:name ?artwork_name.  
      ?artwork pgt:conservationState "Mutilo"^^xsd:string .  
    }   
    GROUP BY (?site) HAVING( ?artwork_count > 50 )  
    ORDER BY DESC (?artwork_count)  
''')

1


[[('site', 'pgt:SITE1'),
  ('artwork_count', '95'),
  ('artwork_names',
   "Kouros, Guerriero in assalto, Guerriero in assalto, Kore, Ercole in assalto, Guerriero in assalto, Ercole in assalto, Ercole in assalto, Kore, Ercole in assalto, Kore, Ercole in assalto, Guerriero a cavallo, Orante, Ercole in assalto, Cavaliere, Orante, Apollo con patera, Cavaliere, Viaggio agli Inferi, Scena di commiato, Uccelli su albero, Fannii, Bollo CARTORIAN, Ratto di Ganimede, Bollo CARTORIAN, Motivi geometrici e fitomorfi, Sileno, Venere che si toglie il sandalo, Iscrizione di Marco Arrunzio Aquila, Atto evergetico di Allenius Strabo, Statua femminile acefala, Giove tonante, Mercurio, Iscrizione di Tilonia Musice, Iscrizione, Iscrizione Rutilius Regillus e Lucius Porcius Germanus, Iscrizione di C. Arrius Optatus, A scorrimento con testa ad anello, Iscrizione, Iscrizione di Quinto Satrio Severo, Iscritta, Bollo CLAVD PANSI, Ritratto femminile, Due busti e un bambino, Iscrizione di Q. Pomponius Severus, R

## 10 -Authors that painted the biggest paintings

In [ ]:
query(''' 
SELECT ?author ?paintingName (?height*?width as ?surface) 
WHERE { 
    ?artwork a pgt:Artwork; 
            sdo:author ?author; 
            sdo:name ?paintingName; 
            pgt:tag ?tag; 
            pgt:height ?height; 
            pgt:width ?width. 
    FILTER(REGEX(?tag,"dipinto")) 
} 
ORDER BY DESC(?surface) 
LIMIT 10 
''')

10


[[('author', 'Jacopo Negretti detto Palma il Giovane'),
  ('paintingName',
   'Dipinto celebrativo dei rettori Jacopo e Giovanni Soranzo'),
  ('surface', '128760.0')],
 [('author', 'Girolamo da Romano detto Romanino'),
  ('paintingName',
   'Madonna con il Bambino e i santi Benedetto, Giustina, Prosdocimo e Scolastica\ncimasa: Pietà\ntondi: san Luca, san Mattia, san Massimo e san Giuliano da Padova e tre santi martiri innocenti'),
  ('surface', '104800.0')],
 [('author', 'Giambattista Tiepolo'),
  ('paintingName', "San Patrizio vescovo d'Irlanda"),
  ('surface', '56088.0')],
 [('author', 'Giambattista Piazzetta'),
  ('paintingName', 'Cena in Emmaus'),
  ('surface', '55692.0')],
 [('author', 'Jacopo Robusti detto Tintoretto'),
  ('paintingName', 'Crocifissione'),
  ('surface', '53530.0')],
 [('author', 'Ludovico Gallina'),
  ('paintingName', 'Ritratto del doge Paolo Renier'),
  ('surface', '45264.0')],
 [('author', 'Giotto'), ('paintingName', 'Croce'), ('surface', '36572.0')],
 [('autho